In [1]:
# import the required libraries
import docx2txt
import os
import re
import csv
import pandas as pd
import numpy as np

In [2]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = 'C://temp//digidure//05062023//'
input_file = os.path.join(folderlink, 'Repertoriummetoudepersoonsnummers.docx')
output_txt = folderlink+'output.txt'
output_csv = folderlink+'output_file.csv'
folder_output = 'output//'


In [3]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Converting the docx to a text file and remove all unecessary rows.

# Use docx2txt library to extract text from .docx file
text = docx2txt.process(input_file)

# Remove excessive whitespaces
data = ' '.join(text.split())

 "N.N. "de oude vicarius">pred. Lichtenvoorde 1602 tot 1615.<20871>" has been updated to 
 "N.N. "de oude vicarius" pred. Lichtenvoorde 1602 tot 1615.<20871>" manually, since it was the only record that had a > symbol in the line.
 
"Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713,;overl. 28 maart 1715.<1185>"

Was updates to:

Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713, overl. 28 maart 1715.<1185>

Since it splitted the string based on ",;overl"
 

In [5]:
data = data.replace('overl. 28 maart 1715.<1185>', ' overl. 28 maart 1715.<1185>').replace('>pred.', ' pred.')

In [6]:
# Replace semicolons with newlines and add semicolons around < and > since these identify the IDs
data = data.replace(';', ';\n').replace(';\n ', '; ').replace('>', '>;\n ').replace('<', ';<')
lines = data.split('\n')

lines = [line for line in lines if not line.startswith('; ;') and not line.startswith('; ')]
data = '\n'.join(lines)
lines = data.strip().split('\n')
data = '\n'.join([line.lstrip() for line in lines])

In [7]:
with open(output_txt, 'w', encoding='utf-8') as f:
    f.write(data)

In [8]:
columns = ("Geb.", # deze is cruciaal
 " pred.",
 "overl.", # deze is cruciaal
 "Gedoopt", # deze is cruciaal
 "legerpred.",
 "pastoor",
 "garnizoenspred.",
 "emer.",
 "begraven",
 "conrector",
 " rector",
 "monnik",           
 "schoolmeester",
 "hoogleraar",
 "chirurgijn",
 "praeceptor",
 "ziekentrooster",
 "vlootpred.",
 "legerpred.",
 "ambassadepred."
           
)          
    

In [9]:
for column in columns:
    with open(output_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()        
    with open(output_txt,'w', encoding='utf-8') as f:
        for line in lines:
            if "; "+column in line:
                f.write(line)
            elif column in line:
                line = line.replace(column, ";"+column)
                f.write(line)
            else:
                f.write(line)

In [10]:
# Define the headers for the output file
headers = ['surname', 'name', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7', 'Field8', 'Field9', 'Field10', 'Field11','Field12','Field13','Field14','Field15','Field16','Field17']

with open(output_txt, 'r', encoding='utf-8') as infile, open(output_csv, 'w', newline='', encoding='utf-8' ) as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')
    
    # Write the headers to the output file
    writer.writerow(headers)

    # Loop through each row in the input file and write it to the output file with 12 fields
    for row in reader:
        # Create a new row with 12 fields by extending the current row with empty values
        new_row = row + [''] * (12 - len(row))
        writer.writerow(new_row)

In [11]:
df = pd.read_csv(output_csv, sep=';', encoding='utf-8')

In [12]:
#In the file all IDs are stored between < and > therefore 
for column in df.columns:
    # Check if any value in the column contains '<'
    if df[column].astype(str).str.contains('<').any():
        # Copy the values containing '<' to the 'test' column
        df.loc[df[column].astype(str).str.contains('<'), 'id'] = df[column]


In [13]:
df['id'] = df['id'].str.replace('>', '').str.replace('<','')

In [14]:
for column in columns:
    df[column] = df.apply(lambda row: row[row.astype(str).str.contains(column)].iloc[0] if any(row.astype(str).str.contains(column)) else '', axis=1)

In [15]:
df['original_input'] = df['surname'].fillna('') + df['name'].fillna('') + df['Field1'].fillna('') + df['Field2'].fillna('')+ df['Field3'].fillna('')+ df['Field4'].fillna('')+ df['Field5'].fillna('')+ df['Field6'].fillna('')+ df['Field7'].fillna('')+ df['Field8'].fillna('')+ df['Field9'].fillna('')+ df['Field10'].fillna('')+ df['Field11'].fillna('')+ df['Field12'].fillna('')+ df['Field13'].fillna('')+ df['Field14'].fillna('')+ df['Field15'].fillna('')+ df['Field16'].fillna('')+ df['Field17'].fillna('')

In [16]:
for column in columns:
    df[column] = df[column].str.replace(column, '')

In [17]:
df['name_info_father'] =df['name'] .str.extract(r'\[(.*?)\]')

In [18]:
columns_rename = {
    'Geb.': 'birth', 
    ' pred.': 'minister', 
    'overl.': 'death', 
    'Gedoopt':'baptized', 
    'legerpred.':'legerpredikant',
    'pastoor':'pastoor',
    'garnizoenspred.':'garnizoenspredikant',
    "emer.":'emeritus_status',
    "begraven":'burried',
    "conrector":'conrector',
    " rector":'rector',
    "monnik":'monnik',           
    "schoolmeester":'schoolmeester',
    "hoogleraar":'hoogleraar',
    "chirurgijn":'chirurgijn',
    "praeceptor":'praeceptor',
    "ziekentrooster":'ziekentrooster',
    "vlootpred.":'vlootpredikant',
    "legerpred.":'legerpredikant',
    "ambassadepred.":'ambassadepredikant'}

In [19]:
# Rename the columns
df = df.rename(columns=columns_rename)
new_columns = list(columns_rename.values())


In [20]:
df.head(50)

,surname,name,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Field12,Field13,Field14,Field15,Field16,Field17,id,birth,minister,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_father
0,"Aalburg, van",Johannes,Geb. Zierikzee ca. 1717,NaN,"pred. Oudkarspel 30 juni 1743,",overl. 14 maart 1777.,<1>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Zierikzee ca. 1717,"Oudkarspel 30 juni 1743,",14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN
1,Aalst,Wilhelmus,Gedoopt Biggekerke 5 jan. 1664,NaN,"pred. Aardenburg 22 mei 1695,",overl. 19 dec. 1700.,<4>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,,"Aardenburg 22 mei 1695,",19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN
2,"Aalst, van",Cornelius,Geb. Castricum ca. 1686,ambassadepred. in Parijs maart tot dec. 1715,NaN,"pred. Kalslagen ber. 21 febr. 1717,",emer. 1751,overl. Amsterdam 27 aug. 1756.,<2>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Castricum ca. 1686,"Kalslagen ber. 21 febr. 1717,",Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN
3,"Aalst, van",Gerardus,Geb. xxx sept. 1678,NaN,"pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",emer. 1755,overl. 29 juni 1759.,<3>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,xxx sept. 1678,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN
4,Aalstius,Henricus,Geb. 's-Hertogenbosch (?) yyy,NaN,"pred. Castricum en Heemskerk nov. 1700,",emer. sept. 1733,overl. Limmen 15 maart 1736.,<5>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,'s-Hertogenbosch (?) yyy,"Castricum en Heemskerk nov. 1700,",Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN
5,Aalstius,Johannes,Geb. xxx yyy,NaN,"pred. Isabella en Anthonyschans 1642, Oirschot en Best 13 sept. 1648,",overl. 11 dec. 1657.,<7>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,xxx yyy,"Isabella en Anthonyschans 1642, Oirschot en Best 13 sept. 1648,",11 dec. 1657.,,,,,,,,,,,,,,,,,"Aalstius Johannes Geb. xxx yyy pred. Isabella en Anthonyschans 1642, Oirschot en Best 13 sept. 1648, overl. 11 dec. 1657.<7>",NaN
6,Aalstius,Johannes [z.v. Johannes Leonardusz.],Geb. xxx 1620,NaN,"pred. Obdam 1644, Boxtel 13 sept. 1648, Bergen op Zoom 9 nov. 1652 (tevens Waals","pred.),",emer. 1685,begraven 17 nov. 1687.,<6>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,xxx 1620,"Obdam 1644, Boxtel 13 sept. 1648, Bergen op Zoom 9 nov. 1652 (tevens Waals",,,,,,1685,17 nov. 1687.,,,,,,,,,,,"Aalstius Johannes [z.v. Johannes Leonardusz.] Geb. xxx 1620 pred. Obdam 1644, Boxtel 13 sept. 1648, Bergen op Zoom 9 nov. 1652 (tevens Waals pred.), emer. 1685 begraven 17 nov. 1687.<6>",z.v. Johannes Leonardusz.
7,Aalstius,Johannes,Geb. 's-Hertogenbosch ca. 1660,NaN,"pred. Hoornaar 1687, Beverwijk april 1698, Middelburg 22 nov. 1705 (tevens","hoogleraar 1705 tot 1709),",overl. 10 juni 1712.,<8>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,'s-Hertogenbosch ca. 1660,"Hoornaar 1687, Beverwijk april 1698, Middelburg 22 nov. 1705 (tevens",10 juni 1712.,,,,,,,,,,,"1705 tot 1709),",,,,,,"Aalstius Johannes Geb. 's-Hertogenbosch ca. 1660 pred. Hoor

In [21]:
array_drop = [i for i in range(1, 18)]
for dropid in array_drop:
    column_dropid = 'Field'+str(dropid)
    df = df.drop(column_dropid, axis=1)

In [22]:
def extract_year(text):
    match = re.search(r'\d{4}', text)
    if match:
        return match.group(0)
    else:
        return None

In [23]:
function_year = [word for word in new_columns if word != 'minister']

print(function_year)

['birth', 'death', 'baptized', 'legerpredikant', 'pastoor', 'garnizoenspredikant', 'emeritus_status', 'burried', 'conrector', 'rector', 'monnik', 'schoolmeester', 'hoogleraar', 'chirurgijn', 'praeceptor', 'ziekentrooster', 'vlootpredikant', 'ambassadepredikant']


In [24]:
df.head()

,surname,name,id,birth,minister,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_father
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,"Oudkarspel 30 juni 1743,",14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN
1,Aalst,Wilhelmus,4,,"Aardenburg 22 mei 1695,",19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,"Kalslagen ber. 21 febr. 1717,",Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN
3,"Aalst, van",Gerardus,3,xxx sept. 1678,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,"Castricum en Heemskerk nov. 1700,",Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN


In [25]:
for year in function_year:
    fld_year = 'year_' +year
    df[fld_year] = df[year].apply(lambda x: extract_year(x))    

In [26]:
for year_accu in function_year:
    accu_fld_year = 'accu_year_' +year_accu
    df[accu_fld_year] = ''
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the string contains "ca." (case-insensitive)
        if 'ca.' in row[year_accu].lower():
            # If found, set the value of the "accuracy" column to "circa"
            df.at[index, accu_fld_year] = 'circa'
            
    

In [27]:
df.head(100)


,surname,name,id,birth,minister,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_father,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,"Oudkarspel 30 juni 1743,",14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,"Aardenburg 22 mei 1695,",19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,"Kalslagen ber. 21 febr. 1717,",Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,
3,"Aalst, van",Gerardus,3,xxx sept. 1678,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,"Castricum en Heemskerk nov. 1700,",Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
5,Aalstius,Johannes,7,xxx yyy,"Isabella en Anthonyschans 1642, Oirschot en Best 13 sept. 1648,",11 dec. 1657.,,,,,,,,,,,,,,,,,"Aalstius Johannes Geb. xxx yyy pred. Isabella en Anthonyschans 1642, Oirschot en Best 13 sept. 1648, overl. 11 dec. 1657.<7>",NaN,None,1657,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
6,Aalstius,Johannes [z.v. Johannes Leonardusz.],6,xxx 1620,"Obdam 1644, Boxtel 13 sept. 1648, Bergen op Zoom 9 nov. 1652 (tevens Waals",,,,,,1685,17 nov. 1687.,,,,,,,,,,,"Aalstius Johannes [z.v. Johannes Leonardusz.] Geb. xxx 1620 pred. Obdam 1644, Boxtel 13 sept. 1648, Bergen op Zoom 9 nov. 1652 (tevens Waals pred.), emer. 1685 begraven 17 nov. 1687.<6>",z.v. Johannes Leonardusz.,1620,None,None,None,None,None,1685,1687,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
7,Aalstius,Johannes,8,'s-Hertogenbosch ca. 1660,"Hoornaar 1687, Beverwijk april 1698, Middelburg 22 nov. 1705 (tevens",10 juni 1712.,,,,,,,,,,,"1705 tot 1709),",,,,,,"Aalstius Johannes Geb. 's-Hertogenbosch ca. 1660 pred. Hoornaar 1687, Beverwijk april 1698, Middelburg 22 nov. 1705 (tevens hoogl

In [28]:
months =(" januari ", 
 " februari ",
 " maart ",
 " april ",
 " mei ",
 " juni ",
 " juli ",
 " augustus ",
 " september ",
 " oktober ",
 " november ",           
 " december ",
 "jan. ",
 "feb. ",
 "mrt. ",
 "apr. ",
 "jun. ",
 "jul. ",
 "aug. ",
 "sept. ",
 "sep. ",
 "okt. ",
 "nov. ",
 "dec. ",
 "yyy",
 "xxx",
 "ca.",
 "febr."        
) 


In [29]:
for column_strip in function_year:
    #df[column_strip].str.replace('ca.', '').str.replace('yyy', '').str.replace('xxx', '')
    for month in months:
        df[column_strip] = df[column_strip].str.replace(month, '')


In [30]:
for column_strip in function_year:
    df[column_strip] = df[column_strip].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [31]:
df.head()

,surname,name,id,birth,minister,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_father,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee,"Oudkarspel 30 juni 1743,",,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,"Aardenburg 22 mei 1695,",,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum,"Kalslagen ber. 21 febr. 1717,",Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,
3,"Aalst, van",Gerardus,3,,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),"Castricum en Heemskerk nov. 1700,",Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,


In [32]:
birth_org = df[['id','birth','year_birth','accu_year_birth']]
birth_org.to_csv(folderlink+folder_output+'birth_info.csv', sep=';', encoding='utf-8', index=False)


In [33]:
death_org = df[['id','death','year_death','accu_year_death']]
death_org.to_csv(folderlink+folder_output+'death_info.csv', sep=';', encoding='utf-8', index=False)

In [34]:
bapt_org = df[['id','baptized','year_baptized','accu_year_baptized']]
bapt_org.to_csv(folderlink+folder_output+'bapt_info.csv', sep=';', encoding='utf-8', index=False)

In [35]:
df.to_csv(folderlink+folder_output+'parent_data.csv', sep=';', encoding='utf-8', index=False)

From here we are going to make the parent child relationship with predikanten Ministers

Here the minister data is set as a child. 

In [36]:
subset_pred = df[['id','minister']]

In [37]:
subset_pred.head()

,id,minister
0,1,"Oudkarspel 30 juni 1743,"
1,4,"Aardenburg 22 mei 1695,"
2,2,"Kalslagen ber. 21 febr. 1717,"
3,3,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,"
4,5,"Castricum en Heemskerk nov. 1700,"


In [40]:
df_expanded = subset_pred.assign(minister=subset_pred['minister'].str.split(','))

# Explode the 'pred.' column to create separate rows for each item
df_expanded = df_expanded.explode('minister')

In [41]:
df_expanded.head()

,id,minister
0,1,Oudkarspel 30 juni 1743
0,1,
1,4,Aardenburg 22 mei 1695
1,4,
2,2,Kalslagen ber. 21 febr. 1717


In [42]:
df_filtered = df_expanded[['id','minister']]

In [43]:
childs = df_filtered[df_filtered["minister"] !=" "]

In [44]:
childs.head()

,id,minister
0,1,Oudkarspel 30 juni 1743
1,4,Aardenburg 22 mei 1695
2,2,Kalslagen ber. 21 febr. 1717
3,3,Vuren en Dalem 10 aug. 1704
3,3,Sommelsdijk 13 juni 1706


In [45]:
childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))

C:\Windows\Temp\ipykernel_9572\3394215768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))


In [46]:
accu_fld_year = 'accu_year_' +'minster'
childs[accu_fld_year] = ''
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the string contains "ca." (case-insensitive)
    if 'ca.' in row[year_accu].lower():
        # If found, set the value of the "accuracy" column to "circa"
        df.at[index, accu_fld_year] = 'circa'

C:\Windows\Temp\ipykernel_9572\322161388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs[accu_fld_year] = ''


In [47]:
childs.head()

,id,minister,minister_year,accu_year_minster
0,1,Oudkarspel 30 juni 1743,1743,
1,4,Aardenburg 22 mei 1695,1695,
2,2,Kalslagen ber. 21 febr. 1717,1717,
3,3,Vuren en Dalem 10 aug. 1704,1704,
3,3,Sommelsdijk 13 juni 1706,1706,


In [48]:
for month in months:
        childs['minister'] = childs['minister'].str.replace(month, '')

childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))

C:\Windows\Temp\ipykernel_9572\1973265549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].str.replace(month, '')
C:\Windows\Temp\ipykernel_9572\1973265549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))


In [49]:
childs.head()

,id,minister,minister_year,accu_year_minster
0,1,Oudkarspel,1743,
1,4,Aardenburg,1695,
2,2,Kalslagen ber febr,1717,
3,3,Vuren en Dalem,1704,
3,3,Sommelsdijk,1706,


In [50]:
childs.to_csv(folderlink+folder_output+'minister_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
#import the geonames file for the Netherlands

gf= pd.read_csv(folderlink+'geonl_sel.csv')

In [ ]:
pred_child_geo = childs.merge(gf, left_on=childs["minister"].str.extract(f"({'|'.join(gf['placename'])})")[0], right_on="placename", how="inner")

In [ ]:
pred_child_geo.to_csv(folderlink+folder_output+'minister_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
sorted_gf = gf.sort_values(by='placename', key=lambda x: x.str.len(), ascending=False)
sorted_gf = sorted_gf.reset_index(drop=True)

In [ ]:
birth_geo = birth_org.merge(sorted_gf, left_on=df["birth"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
birth_geo.to_csv(folderlink+folder_output+'birth_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
death_geo = death_org.merge(sorted_gf, left_on=df["death"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
death_geo.to_csv(folderlink+folder_output+'death_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
bapt_geo = bapt_org.merge(sorted_gf, left_on=df["baptized"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
bapt_geo.to_csv(folderlink+folder_output+'bapt_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
os.remove(output_txt)
os.remove(output_csv)